<a href="https://colab.research.google.com/github/HSS107048212/Neil/blob/main/Neil_GATcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
import os

In [ ]:
patent_data = pd.read_excel('c9-2.xlsx', header=1) #這一個要放蒐集下來的核心專利
patent_data_family = pd.read_excel('c9-family.xlsx', header=1) #這一個要放核心專利的專利家族和集
patent_data = patent_data.fillna('')

#下面是將專利家族的引用關係複製到核心專利那個上面，由於在建構引用網路時重複的關係並不會被計算到，因此採直接複製
#下面這段是在複製DWPI family的資料

for i in patent_data.index: #整理 Cited Refs - Patent 欄位
    if pd.notnull(patent_data.loc[i,'DWPI Family Members']):
        temp = patent_data.loc[i,'DWPI Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in patent_data_family['Publication Number'].values:
                patent_index = patent_data_family.index[patent_data_family['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Cited Refs - Patent'][i]
                    b = patent_data_family['Cited Refs - Patent'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Cited Refs - Patent'][i] = c

for i in patent_data.index: # 整理 Citing Patents 欄位
    if pd.notnull(patent_data.loc[i,'DWPI Family Members']):
        temp = patent_data.loc[i,'DWPI Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in patent_data_family['Publication Number'].values:
                patent_index = patent_data_family.index[patent_data_family['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Citing Patents'][i]
                    b = patent_data_family['Citing Patents'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Citing Patents'][i] = c


#下面是做一樣的事情，只是變成複製INPADC的資料，單純因為懶，所以分成兩個比較方便
INPADC1 = pd.read_excel('INPADC1.xlsx', header=1)
INPADC2 = pd.read_excel('INPADC2.xlsx', header=1)
INPADC3 = pd.read_excel('INPADC3.xlsx', header=1)
INPADC4 = pd.read_excel('INPADC4.xlsx', header=1)
INPADC = pd.concat([INPADC1,INPADC2])
INPADC = pd.concat([INPADC,INPADC3])
INPADC = pd.concat([INPADC,INPADC4])
INPADC.index = list(range(len(INPADC)))

for i in patent_data.index:
    if pd.notnull(patent_data.loc[i,'DWPI Family Members']):
        temp = patent_data.loc[i,'DWPI Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in INPADC['Publication Number'].values:
                patent_index = INPADC.index[INPADC['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Cited Refs - Patent'][i]
                    b = INPADC['Cited Refs - Patent'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Cited Refs - Patent'][i] = c

for i in patent_data.index:
    if pd.notnull(patent_data.loc[i,'INPADOC Family Members']):
        temp = patent_data.loc[i,'INPADOC Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in INPADC['Publication Number'].values:
                patent_index = INPADC.index[INPADC['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Citing Patents'][i]
                    b = INPADC['Citing Patents'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Citing Patents'][i] = c

#最後將引用關係合併之後的資料集輸出為excel，在後續分析時要用
patent_data.to_excel('c9-append.xlsx')

<ipython-input-4-b012f9d53a5c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patent_data['Cited Refs - Patent'][i] = c
<ipython-input-4-b012f9d53a5c>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patent_data['Citing Patents'][i] = c


In [ ]:
patent_data

,Publication Number,Title (English),Assignee/Applicant,Publication Year,Publication Date,Abstract (English),Abstract - DWPI,Cited Refs - Patent,Citing Patents,Count of Citing Patents,Count of Cited Refs - Patent,IPC Current Full (4 Characters),INPADOC Family Members,DWPI Family Members
0,TW201818743A,ROUTING METHOD AND WIRELESS NODE FOR WIRELESS ...,"INDUSTRIAL TECHNOLOGY RESEARCH INSTITUTE,TW | ...",2018,2018-05-16,A routing method for a wireless mesh network a...,The method involves performing (S610) route al...,US7159035B2 | US8488589B2 | US8948046B2 | US90...,CN113259940A | CN113259940B | TWI727786B|CN114...,3,4,"H04L, H04W",TW201818743A | TWI604744B | US10250503B2 | US...,US20180123957A1 | TWI604744B | TW201818743A |...
1,TW201946421A,TRANSPARENT BRIDGING OVER MESH NETWORK,"GUIZHOU DINGTONGXIN TECHNOLOGIES LTD.,CN | 大陸商...",2019,2019-12-01,A method of transparent bridging over mesh net...,The method involves receiving a packet from an...,US20170034760A1 | US8730809B2 | US20130100872A...,,0,4,H04L,TW201946421A | CN110417659A | TWI731237B,CN110417659A | TW201946421A | TWI731237B
2,TW201715902A,MANAGEMENT METHOD FOR WIRELESS NETWORK AND NET...,"INDUSTRIAL TECHNOLOGY RESEARCH INSTITUTE,TW | ...",2017,2017-05-01,A wireless network management method for a wir...,A wireless network management method for a wir...,US20110255570A1 | US20140153551A1 | US20140119...,WO2019127204A1|WO2019127204A1|US10356759B2 | U...,1,3,H04W,TW201715902A | TWI568279B | US20170118752A1 |...,TW201715902A | TWI568279B | US9999045B2 | US2...
3,TW202245502A,METHOD AND SYSTEM FOR UPDATING DEVICE FIRMWARE...,"COMPAL BROADBAND NETWORKS INC.,TW | 鋐寶科技股份有限公司,TW",2022,2022-11-16,A firmware update method. On a wireless mesh n...,A firmware update method. On a wireless mesh n...,CN106713047A | CN102158851B | CN103685530A|CN1...,,0,3,"H04W, H04L, G06F",,TW202245502A | TWI784506B
4,TW202245501A,METHOD AND SYSTEM FOR REMOTELY CONTROLLING FIR...,"COMPAL BROADBAND NETWORKS INC.,TW | 鋐寶科技股份有限公司,TW",2022,2022-11-16,A method or system for remotely controlling th...,A method or system for remotely controlling th...,CN103685530A | CN106713047A | CN102158851B|CN1...,,0,3,"H04W, H04L, G06F",,TW202245501A | TWI784505B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,TW202114382A,ROUTER AND ROUTING METHOD,"REALTEK SEMICONDUCTOR CORPORATION,TW | 瑞昱半導體股份...",2021,2021-04-01,The application discloses a router coupled to ...,The router (100) has a switch module (112) pro...,US8565240B2 | US20030016628A1 | WO2013081511A1...,,0,4,H04L,TW202114382A | TWI730476B | US11206568B2 | US2...,US20210092633A1 | TW202114382A | TWI730476B | ...
92,TW202113749A,SYSTEM FOR INTEGRATED MANAGEMENT OF VIRTUAL AN...,"CHUNGHWA TELECOM CO. LTD.,TW | 中華電信股份有限公司,TW",2021,2021-04-01,The present invention is a system for integrat...,The present invention is a system for integrat...,TW201527991A | TW201911045A | TW200611112A | T...,,0,5,"G06Q, H04L",TW202113749A | TWI709941B,TWI709941B | TW202113749A
93,TW202121869A,MANAGEMENT SYSTEM AND MANAGEMENT METHOD OF VNF,"CHUNGHWA TELECOM CO. LTD.,TW | 中華電信股份有限公司,TW",2021,2021-06-01,A management system and management method of V...,A management system and management method of V...,TW201909601A | WO2015106822A1 | US20190058636A...,|TWI789014B | TWI795123B,0,7,H04L,TW202121869A | TWI707561B,TWI707561B | TW202121869A
94,TW202121917A,CHANNEL LOADING PRE-ADJUSTING SYSTEM FOR 5G WI...,"ALPHA NETWORKS INC.,TW | 明泰科技股份有限公司,TW",2021,2021-06-01,A channel loading pre-adjusting system for 5G ...,The channel loading pre-adjusting system (10) ...,TW201212693A | US20170332270A1 | US20080134194...,|CN113965946A,0,3,H04W,TW202121917A | CN112839357A | CN112839357B | T...,US20210160734A1 | CN112839357A | TWI701956B | ...


In [ ]:
import pandas as pd
import numpy as np
import copy

import os

#從Derwent下載下來的資料，核心專利、核心專利引用的專利跟引用核心專利的專利
#patent_data = pd.read_excel('1215_append.xlsx', header=0) #這個是要使用合併引用關係之後的核心資料集
#cited_data = pd.read_excel('1215_cited.xlsx', header=1)
#citing_data = pd.read_excel('1215_citing.xlsx', header=1)

# Neil學長的 已经将Excel 文件转换为了 CSV 格式，并且文件名保持一致，只是扩展名变为了 .csv
patent_data = pd.read_csv('1215_append.csv', header=0, encoding='utf-8') # 这个是要使用合并引用关系之后的核心数据集
cited_data = pd.read_csv('1215_cited.csv', header=2, encoding='utf-8')
citing_data = pd.read_csv('1215_citing.csv', header=2, encoding='utf-8')

#cited_data = cited_data.iloc[3:]
#citing_data = citing_data.iloc[3:]
print(patent_data.columns)
print(cited_data.columns)
print(citing_data.columns)


#整理資料，保留特定欄位並合併三個資料集為全部使用到的大資料集。
temp_patent = patent_data[['Publication Number','Title (English)','Assignee/Applicant','Publication Year','Publication Date','Abstract (English)','Abstract - DWPI','Cited Refs - Patent','Citing Patents','Count of Cited Refs - Patent','Count of Citing Patents'] ]
temp_cited = cited_data[['Publication Number','Title (English)','Assignee/Applicant','Publication Year','Publication Date','Abstract (English)','Abstract - DWPI','Cited Refs - Patent','Citing Patents','Count of Cited Refs - Patent','Count of Citing Patents']]
temp_citing = citing_data[['Publication Number','Title (English)','Assignee/Applicant','Publication Year','Publication Date','Abstract (English)','Abstract - DWPI','Cited Refs - Patent','Citing Patents','Count of Cited Refs - Patent','Count of Citing Patents']]
all_patent = pd.concat([temp_patent,temp_cited])
all_patent = pd.concat([all_patent,temp_citing])
all_patent = all_patent.dropna(subset=['Title (English)','Abstract (English)','Abstract - DWPI']) #將三行都是空值的列刪除
#刪除重複值並用Publication Year排序
all_patent = all_patent.drop_duplicates()

# 将 'Publication Year' 列转换为整数，无法转换的设置为 NaN
all_patent['Publication Year'] = pd.to_numeric(all_patent['Publication Year'], errors='coerce')

# 现在可以排序了，因为所有数据都是整数类型
all_patent = all_patent.sort_values('Publication Year')

# 如果你想要删除 'Publication Year' 列中的 NaN 值
all_patent = all_patent.dropna(subset=['Publication Year'])

all_patent = all_patent.sort_values('Publication Year')
all_patent1 = all_patent.drop_duplicates(subset=['Publication Number'])
all_patent1.index = list(range(len(all_patent1)))
#合併會使用到的專利文字資料，並在原資料及創造一個新的欄位名叫text存放
text = []
for i in all_patent1.index:

  str1 = str(all_patent1.loc[i,'Title (English)'])+' '+str(all_patent1.loc[i,'Abstract (English)'])+' '+str(all_patent1.loc[i,'Abstract - DWPI'])

  str2 = ''.join(str1.split('nan'))
  text.append(str2)
text = np.array(text)
all_patent1['text'] = text

#有text的專利資料集先存起來備用
all_patent1.to_excel('IoV new.xlsx')

Index(['Unnamed: 0', 'Publication Number', 'Title (English)',
       'Assignee/Applicant', 'Publication Year', 'Publication Date',
       'Abstract (English)', 'Abstract - DWPI', 'Cited Refs - Patent',
       'Citing Patents', 'Count of Citing Patents',
       'Count of Cited Refs - Patent', 'IPC Current Full (4 Characters)',
       'INPADOC Family Members', 'DWPI Family Members'],
      dtype='object')
Index(['Publication Number', 'Title (English)', 'Assignee/Applicant',
       'Publication Year', 'Publication Date', 'Abstract (English)',
       'Abstract - DWPI', 'Cited Refs - Patent', 'Citing Patents',
       'Count of Citing Patents', 'Count of Cited Refs - Patent',
       'IPC Current Full (4 Characters)', 'INPADOC Family Members',
       'DWPI Family Members'],
      dtype='object')
Index(['Publication Number', 'Title (English)', 'Assignee/Applicant',
       'Publication Year', 'Publication Date', 'Abstract (English)',
       'Abstract - DWPI', 'Cited Refs - Patent', 'Citing Pate

<ipython-input-1-179338ce6ccf>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_patent['Publication Year'] = pd.to_numeric(all_patent['Publication Year'], errors='coerce')


20231227test

In [3]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=dcdf41603519ad3df9d46560481d888b9cfd175bb988d142d2b89235e80b55a9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
pip install corextopic

In [2]:
import pandas as pd
import numpy as np  # 需要導入 NumPy 库
# 指定 Excel 檔案的路徑
excel_file_path = 'IoV new.xlsx'

# 讀取 Excel 檔案
df = pd.read_excel(excel_file_path)
text = np.array(df['text'])

In [ ]:
#Sentence Transformer 將文字轉成向量，paraphrase-distilroberta-base-v1為在英文上表現最好BERT的模型
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
text_embeddings = model.encode(text)
text_embeddings_df = pd.DataFrame(text_embeddings)
#將文字向量存起來，之後就不需要再跑第二次
text_embeddings_df.to_csv('text_embeddings_title_abstract_new.csv',index=False)

In [4]:
#-------------------------------------------------------------------------------接下來是進行主題模型
import  scipy.sparse as ss
import matplotlib.pyplot as plt
import corextopic.corextopic as ct
import corextopic.vis_topic as vt
from sklearn.feature_extraction.text import CountVectorizer

ModuleNotFoundError: ignored

In [ ]:
#讀取存有專利文字資料的資料集
patents = pd.read_excel('IoV new.xlsx')

In [ ]:
text = patents['text'].values
#先刪除一些不在常規停用字的停用字，這邊可以自由新增減少，從關鍵字的結果中可以判斷，通常專利中有很多垃圾字，需要先處理過後再跑主題模型會好很多。
stopwords = ['aspects of subject','non-English','non english','flow chart','block diagram','copyright', 'jpo', 'japio','100wireless','Drawing','includes','language','text','drawing','claims','claim','following','english','aa','shows','figure','method','methods','flowchart',
             'illustrating','involves','diagram','schematic','block','chart','explanatory view','representation','determined','determining','realize','realizing','invention','according','technical','comprises','field','realizes','problem','problems','solves','module','solving','solves','embodiment',
             'described','associated','systems','display','allows','providing','including','kipo','copyright']
for i in range(len(text)):
    text[i] = text[i].lower()
    for j in stopwords:
        if j in text[i]:
            temp_text = text[i].split(j)
            text[i] = "".join(temp_text)

#使用CountVectorizer，先將常規的停用字刪除，然後將文字資料轉換成1-gram與2-gram的詞，在這些詞中由出現頻率作為依據取前100000個詞
vectorizer = CountVectorizer(stop_words='english',ngram_range = (1,2),max_features=100000,binary=True)
doc_word = vectorizer.fit_transform(text)
doc_word = ss.csr_matrix(doc_word)
# 提取出現頻率100000個1-gram與2-gram的詞
words = list(np.asarray(vectorizer.get_feature_names()))
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words = [word for ind,word in enumerate(words) if not word.isdigit()]

#使用CorEX模型提取主題，n_hidden為主題數，words生成主題使用到的詞，max_iter為CorEX迭代的次數
topic_num = 25  #生成的主體數，需要自己去調控
topic_model = ct.Corex(n_hidden=topic_num,words = words,max_iter = 200,verbose=False,seed = 2) #這邊是corex的參數，可調可不調，看結果
topic_model.fit(doc_word,words=words)
#每個主題提取前50個字
topics = topic_model.get_topics(n_words=50)
keywords = []
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    temp_str = '{}: '.format(n) + ', '.join(topic_words)
    keywords.append(temp_str)
keywords_prob = []
for i in range(topic_num):
    temp = []
    for j in topic_model.get_topics(topic=i, n_words=50):
        temp.append(str(j))
    keywords_prob.append(temp)
#輸出CorEX主題相關資訊
plt.figure(figsize=(10,5))
plt.bar(range(topic_model.tcs.shape[0]), topic_model.tcs, color='#4e79a7', width=0.5)
plt.xlabel('Topic', fontsize=16)
plt.ylabel('Total Correlation (nats)', fontsize=16);
plt.savefig("mygraph_24_anchor.png")

In [ ]:
#下面的prefix代表了輸出主題模型資訊的資料夾名稱，最好可以改成自己認識的字
vt.vis_rep(topic_model, column_label=words, prefix='new IoV patents 30topics 1')
many_keywords_prob = []
for i in range(topic_num):
    temp = []
    for j in topic_model.get_topics(topic=i, n_words=100):
        temp.append(str(j))
    many_keywords_prob.append(temp)
with open('100topic_keyword_prob_patents.txt','w',encoding='utf8') as f:
   for i in many_keywords_prob:
       f.write(str(i))
       f.write('\n')
   f.close()
topics = topic_model.get_topics(n_words=100)
many_keywords = []
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    temp_str = '{}: '.format(n) + ', '.join(topic_words)
    many_keywords.append(temp_str)

with open('100topic_keyword_patents.txt','w',encoding='utf8') as f:
    for i in many_keywords:
        f.write(i)
        f.write('\n')
    f.close()


In [ ]:
with open('labels.txt','r') as f: #這邊的labels，在輸出主題模型資訊的資料夾中可以找到，代表的是專利與其代表主題的號碼
  label = f.read()
  label = label.split('\n')
  f.close()
topic_labels = []
for i in label[:-1]:
  temp = i.split(',')
  topic_labels.append([all_patent1.loc[int(temp[0]),'Publication Number'],temp[1:]])
topic_labels = np.array(topic_labels)
topic_value = []
for i in np.array(topic_labels[:,1]):
  temp = []
  for j in i:
    temp.append(int(j))
  topic_value.append(temp)
topic_value = np.array(topic_value)
topic_value